In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv(r'/content/drive/MyDrive/Dataset/Intellimark/train.csv')
test = pd.read_csv(r'/content/drive/MyDrive/Dataset/Intellimark/test.csv')

In [ ]:
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [ ]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

# Preprocessing with advanced feature engineering
def preprocess_data(data):
    # Fill missing values
    data['Item_Weight'].fillna(data['Item_Weight'].mean(), inplace=True)
    data['Outlet_Size'].fillna('Unknown', inplace=True)
    data['Item_Visibility'].replace(0, data['Item_Visibility'].mean(), inplace=True)

    # Create Item_Age feature
    data['Item_Age'] = 2024 - data['Outlet_Establishment_Year']
    data.drop(['Outlet_Establishment_Year'], axis=1, inplace=True)

    # Log transformation to normalize skewed features
    data['Item_Visibility_Log'] = np.log1p(data['Item_Visibility'])

    # Polynomial feature: Item_Visibility squared
    data['Item_Visibility_Squared'] = data['Item_Visibility'] ** 2

    # Interaction feature: Combining Item_Weight and Item_Visibility
    data['Item_Weight_Visibility'] = data['Item_Weight'] * data['Item_Visibility']

    return data

# Apply preprocessing
train = preprocess_data(train)
test = preprocess_data(test)


<ipython-input-5-4865f34dfa59>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Item_Weight'].fillna(data['Item_Weight'].mean(), inplace=True)
<ipython-input-5-4865f34dfa59>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplac

In [ ]:
import pandas as pd
import numpy as np

def remove_outliers(dataframe):
    """
    Removes outliers from all numerical columns in a dataframe using the IQR method and quantifies removed outliers.

    Parameters:
    dataframe (pd.DataFrame): The input DataFrame.

    Returns:
    pd.DataFrame: A DataFrame without the outliers.
    dict: A dictionary with column names as keys and the number of outliers removed as values.
    """
    cleaned_df = dataframe.copy()
    outliers_count = {}

    for column in cleaned_df.select_dtypes(include=[np.number]).columns:
        Q1 = cleaned_df[column].quantile(0.25)
        Q3 = cleaned_df[column].quantile(0.75)
        IQR = Q3 - Q1

        # Define bounds for outliers
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Count outliers
        outliers = cleaned_df[(cleaned_df[column] < lower_bound) | (cleaned_df[column] > upper_bound)]
        outliers_count[column] = len(outliers)

        # Filter the data
        cleaned_df = cleaned_df[(cleaned_df[column] >= lower_bound) & (cleaned_df[column] <= upper_bound)]

    return cleaned_df, outliers_count


In [ ]:
train, outliers_count = remove_outliers(train)
print("\nDataFrame after removing outliers:")

print("\nNumber of outliers removed:")
for column, count in outliers_count.items():
    print(f"{column}: {count}")



DataFrame after removing outliers:

Number of outliers removed:
Item_Weight: 0
Item_Visibility: 173
Item_MRP: 0
Item_Outlet_Sales: 185
Item_Age: 0
Item_Visibility_Log: 0
Item_Visibility_Squared: 658
Item_Weight_Visibility: 200


In [ ]:
train.isna().sum()

,0
Item_Identifier,0
Item_Weight,0
Item_Fat_Content,0
Item_Visibility,0
Item_Type,0
Item_MRP,0
Outlet_Identifier,0
Outlet_Size,0
Outlet_Location_Type,0
Outlet_Type,0


In [ ]:
train.shape

(7307, 15)

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer, QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import numpy as np

# Identify categorical and numerical columns
categorical_cols = train.select_dtypes(include=['object']).columns.drop('Item_Identifier')
numerical_cols = train.select_dtypes(include=[np.number]).columns.drop(['Item_Outlet_Sales'])

# Preprocessing pipeline
def preprocess_data(train, test):
    # Categorical encoding with OneHotEncoder
    encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

    # Advanced numerical scaling: PowerTransformer and QuantileTransformer
    num_transformer = QuantileTransformer(output_distribution='normal')

    # Combined transformations for numerical and categorical features
    preprocessor = ColumnTransformer([
        ('num', num_transformer, numerical_cols),
        ('cat', encoder, categorical_cols)
    ])

    # Fit and transform on training data, transform test data
    X_train = preprocessor.fit_transform(train)
    X_test = preprocessor.transform(test)

    return X_train, X_test, preprocessor

# Process data
X, X_test, preprocessor = preprocess_data(train, test)

# Target variable
y = train['Item_Outlet_Sales'].values

# Train-test split with stratification for better balance (if applicable)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Final shape check
print("Train shape:", X_train.shape)
print("Validation shape:", X_val.shape)
print("Test shape:", X_test.shape)


Train shape: (5845, 49)
Validation shape: (1462, 49)
Test shape: (5681, 49)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
import xgboost as xgb



# Hyperparameter Tuning
param_grid = {
    'n_estimators': [500, 1000],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [4, 6, 8]
}

model = xgb.XGBRegressor(random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_root_mean_squared_error', verbose=2)
grid_search.fit(X_train, y_train)

# Evaluate the model
best_model = grid_search.best_estimator_
pred_val = best_model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, pred_val))
print(f'Validation RMSE: {rmse}')

# Predict on Test Data
pred_test = np.maximum(best_model.predict(X_test), 0)

# Prepare submission file
submission = pd.DataFrame({
    'Item_Identifier': test['Item_Identifier'],
    'Outlet_Identifier': test['Outlet_Identifier'],
    'Item_Outlet_Sales': pred_test
})

submission.to_csv('submission__P.csv', index=False)
print('Submission file created!')

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=500; total time=   4.7s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=500; total time=   3.6s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=500; total time=   2.7s
[CV] END .learning_rate=0.01, max_depth=4, n_estimators=1000; total time=   1.3s
[CV] END .learning_rate=0.01, max_depth=4, n_estimators=1000; total time=   4.9s
[CV] END .learning_rate=0.01, max_depth=4, n_estimators=1000; total time=   1.1s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=500; total time=   1.3s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=500; total time=   1.3s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=500; total time=   1.3s
[CV] END .learning_rate=0.01, max_depth=6, n_estimators=1000; total time=   2.3s
[CV] END .learning_rate=0.01, max_depth=6, n_estimators=1000; total time=   2.5s
[CV] END .learning_rate=0.01, max_depth=6, n_est

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import StackingRegressor, VotingRegressor
import xgboost as xgb


# Define Base Models
xgb_model = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.05, max_depth=6, random_state=42)
rf_model = RandomForestRegressor(n_estimators=500, max_depth=8, random_state=42)
ridge_model = Ridge(alpha=1.0)

# Stacking Regressor
stacking_model = StackingRegressor(
    estimators=[('xgb', xgb_model), ('rf', rf_model), ('ridge', ridge_model)],
    final_estimator=xgb.XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=4, random_state=42)
)

# Voting Regressor
voting_model = VotingRegressor([('xgb', xgb_model), ('rf', rf_model), ('ridge', ridge_model)])

# Train Stacking Model
stacking_model.fit(X_train, y_train)
pred_val_stacking = stacking_model.predict(X_val)
rmse_stacking = np.sqrt(mean_squared_error(y_val, pred_val_stacking))
print(f'Stacking Model Validation RMSE: {rmse_stacking}')

# Train Voting Model
voting_model.fit(X_train, y_train)
pred_val_voting = voting_model.predict(X_val)
rmse_voting = np.sqrt(mean_squared_error(y_val, pred_val_voting))
print(f'Voting Model Validation RMSE: {rmse_voting}')

# Predict on Test Data using the best model
final_model = stacking_model if rmse_stacking < rmse_voting else voting_model
pred_test = np.maximum(final_model.predict(X_test), 0)

# Prepare submission file
submission = pd.DataFrame({
    'Item_Identifier': test['Item_Identifier'],
    'Outlet_Identifier': test['Outlet_Identifier'],
    'Item_Outlet_Sales': pred_test
})

submission.to_csv('submissionm2.csv', index=False)
print('Submission file created!')


Stacking Model Validation RMSE: 992.8236154855825
Voting Model Validation RMSE: 993.0028409251046
Submission file created!


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PowerTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import StackingRegressor, VotingRegressor
import xgboost as xgb


xgb_model = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.05, max_depth=6, random_state=42)
rf_model = RandomForestRegressor(n_estimators=500, max_depth=8, random_state=42)
ridge_model = Ridge(alpha=1.0)

# Hyperparameter Tuning for XGB
param_grid = {
    'n_estimators': [500, 1000, 1500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [4, 6, 8]
}
search = GridSearchCV(xgb.XGBRegressor(random_state=42), param_grid, cv=3, scoring='neg_root_mean_squared_error')
search.fit(X_train, y_train)

# Stacking Regressor
stacking_model = StackingRegressor(
    estimators=[('xgb', search.best_estimator_), ('rf', rf_model), ('ridge', ridge_model)],
    final_estimator=xgb.XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=4, random_state=42)
)

# Voting Regressor
voting_model = VotingRegressor([('xgb', search.best_estimator_), ('rf', rf_model), ('ridge', ridge_model)])

# Train Stacking Model
stacking_model.fit(X_train, y_train)
pred_val_stacking = stacking_model.predict(X_val)
rmse_stacking = np.sqrt(mean_squared_error(y_val, pred_val_stacking))
print(f'Stacking Model Validation RMSE: {rmse_stacking}')

# Train Voting Model
voting_model.fit(X_train, y_train)
pred_val_voting = voting_model.predict(X_val)
rmse_voting = np.sqrt(mean_squared_error(y_val, pred_val_voting))
print(f'Voting Model Validation RMSE: {rmse_voting}')

# Predict on Test Data using the best model
final_model = stacking_model if rmse_stacking < rmse_voting else voting_model
pred_test = np.maximum(final_model.predict(X_test), 0)

# Prepare submission file
submission = pd.DataFrame({
    'Item_Identifier': test['Item_Identifier'],
    'Outlet_Identifier': test['Outlet_Identifier'],
    'Item_Outlet_Sales': pred_test
})

submission.to_csv('submissionm3.csv', index=False)
print('Submission file created!')


Stacking Model Validation RMSE: 998.8477701939808
Voting Model Validation RMSE: 972.4336926682971
Submission file created!


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Load the data (assuming X and y are already defined)
# Replace with appropriate dataset loading
# X_train, X_test, y_train, y_test = ...

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Function to build ANN with variable parameters
def build_ann(input_shape, layers, neurons, activation, dropout_rate):
    model = Sequential()
    model.add(Dense(neurons[0], input_shape=(input_shape,), activation=activation))

    for i in range(1, layers):
        model.add(Dense(neurons[i], activation=activation))
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate))

    model.add(Dense(1))  # Output layer (regression)
    return model

# Experimenting with parameters
layer_configs = [2, 3, 4]  # Number of hidden layers
neurons_configs = [
    [64, 32],           # 2 layers
    [128, 64, 32],      # 3 layers
    [256, 128, 64, 32]  # 4 layers
]
activations = ['relu', 'tanh']
optimizers = ['adam', 'rmsprop']
dropout_rates = [0.0, 0.2]
batch_sizes = [32, 64]
epochs = 100

best_rmse = float('inf')
best_model = None

# Grid Search Loop
for layers, neurons in zip(layer_configs, neurons_configs):
    for activation in activations:
        for optimizer in optimizers:
            for dropout_rate in dropout_rates:
                for batch_size in batch_sizes:
                    print(f"Training model with {layers} layers, {neurons}, {activation}, {optimizer}, dropout={dropout_rate}, batch={batch_size}")
                    model = build_ann(X_train_scaled.shape[1], layers, neurons, activation, dropout_rate)
                    model.compile(optimizer=optimizer, loss='mse')

                    # Early stopping to prevent overfitting
                    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

                    # Train the model
                    history = model.fit(X_train_scaled, y_train, validation_data=(X_val_scaled, y_val),
                                        epochs=epochs, batch_size=batch_size, verbose=0, callbacks=[early_stop])

                    # Evaluate on the validation set
                    pred_val = model.predict(X_val_scaled)
                    rmse = np.sqrt(mean_squared_error(y_val, pred_val))
                    print(f"Validation RMSE: {rmse}")

                    # Save the best model
                    if rmse < best_rmse:
                        best_rmse = rmse
                        best_model = model
                        best_params = {
                            'layers': layers,
                            'neurons': neurons,
                            'activation': activation,
                            'optimizer': optimizer,
                            'dropout_rate': dropout_rate,
                            'batch_size': batch_size
                        }

# Final Evaluation on Test Data
print(f"Best RMSE on Validation: {best_rmse}")
print(f"Best Parameters: {best_params}")
pred_test = np.maximum(best_model.predict(X_test_scaled), 0)

# Prepare submission file
submission = pd.DataFrame({
    'Item_Identifier': test['Item_Identifier'],
    'Outlet_Identifier': test['Outlet_Identifier'],
    'Item_Outlet_Sales': pred_test.flatten()
})
submission.to_csv('submission_ann.csv', index=False)
print('Submission file created!')

Training model with 2 layers, [64, 32], relu, adam, dropout=0.0, batch=32


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Validation RMSE: 981.0747668984719
Training model with 2 layers, [64, 32], relu, adam, dropout=0.0, batch=64


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Validation RMSE: 986.2183913471281
Training model with 2 layers, [64, 32], relu, adam, dropout=0.2, batch=32


...

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Validation RMSE: 2331.0090941593344
Training model with 4 layers, [256, 128, 64, 32], tanh, rmsprop, dropout=0.2, batch=32


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Validation RMSE: 2133.6124070290575
Training model with 4 layers, [256, 128, 64, 32], tanh, rmsprop, dropout=0.2, batch=64


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Validation RMSE: 2340.0482076806575
Best RMSE on Validation: 980.7325619727352
Best Parameters: {'layers': 4, 'neurons': [256, 128, 64, 32], 'activation': 'relu', 'optimizer': 'rmsprop', 'dropout_rate': 0.0, 'batch_size': 64}
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Submission file created!
